In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd "/content/gdrive/My Drive/Project/drqa/scripts"

/content/gdrive/My Drive/Project/drqa/scripts


In [0]:
!ls

convert  drqa  Untitled0.ipynb


In [0]:
import re
import json
import spacy
import msgpack
import unicodedata
import numpy as np
import argparse
import collections
import multiprocessing
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial
from drqa.utils import str2bool
import logging


In [0]:
#Creating a class of arguments
class arguments:
  trn_file='/content/gdrive/My Drive/Project/drqa/datasets/squad/train-v1.1.json'
  dev_file='/content/gdrive/My Drive/Project/drqa/datasets/squad/dev-v1.1.json'
  wv_file='/content/gdrive/My Drive/Project/drqa/datasets/glove.840B.300d.txt'
  wv_dim=300
  wv_cased=True
  sort_all=True
  sample_size=0
  threads=2
  batch_size=64

In [0]:
args=arguments()

In [0]:
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG,
                        datefmt='%m/%d/%Y %I:%M:%S')
log = logging.getLogger(__name__)
log.info(vars(args))
log.info('start data preparing...')

04/18/2020 03:19:21 {}
04/18/2020 03:19:21 start data preparing...


Considering only context, question and answer in paragraph in training data. But in dev data there are os may answers wheras in training only one answer.


In [0]:
def flatten_json(data_file, mode):
    """Flatten each article in training data."""
    with open(data_file) as f:
        data = json.load(f)['data']
    rows = []
    for article in data:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                id_, question, answers = qa['id'], qa['question'], qa['answers']
                if mode == 'train':
                    answer = answers[0]['text']  # in training data there's only one answer
                    answer_start = answers[0]['answer_start']
                    answer_end = answer_start + len(answer)
                    rows.append((id_, context, question, answer, answer_start, answer_end))
                else:  # mode == 'dev'
                    answers = [a['text'] for a in answers]
                    rows.append((id_, context, question, answers))
    return rows

In [0]:
train = flatten_json(args.trn_file, 'train')
dev = flatten_json(args.dev_file, 'dev')
log.info('json data flattened.')

04/18/2020 03:24:34 json data flattened.


In [0]:
def clean_spaces(text):
    """normalize spaces in a string."""
    text = re.sub(r'\s', ' ', text)
    return text


def normalize_text(text):
    return unicodedata.normalize('NFD', text) #Decompose if any greek type words are present in text

In [0]:
nlp=None

In [0]:
def init():
    """initialize spacy in each process"""
    global nlp
    nlp = spacy.load('en', parser=False)

In [0]:
def annotate(row, wv_cased):
    global nlp
    id_, context, question = row[:3]
    q_doc = nlp(clean_spaces(question)) #Clean any unnecessary spaces in question
    c_doc = nlp(clean_spaces(context)) #Clean any unnecessary spaces in context
    question_tokens = [normalize_text(w.text) for w in q_doc] #Normalize question tokens
    context_tokens = [normalize_text(w.text) for w in c_doc]  #Normalize context tokens
    question_tokens_lower = [w.lower() for w in question_tokens] #Lower question tokens
    context_tokens_lower = [w.lower() for w in context_tokens] #Lower context tokens
    context_token_span = [(w.idx, w.idx + len(w.text)) for w in c_doc] #Set the pair for span of the word in context words 
    context_tags = [w.tag_ for w in c_doc] #POS tags for context words Eg: NN, VBZ, NNP
    context_ents = [w.ent_type_ for w in c_doc] #NER types for context words Eg:NORP, Work-of-art
    question_lemma = {w.lemma_ if w.lemma_ != '-PRON-' else w.text.lower() for w in q_doc} #Apply Lemmaization for question words
    question_tokens_set = set(question_tokens) 
    question_tokens_lower_set = set(question_tokens_lower)
    match_origin = [w in question_tokens_set for w in context_tokens] #Check if query word is present in context
    match_lower = [w in question_tokens_lower_set for w in context_tokens_lower] #Check if query lower words are present in context lower words
    match_lemma = [(w.lemma_ if w.lemma_ != '-PRON-' else w.text.lower()) in question_lemma for w in c_doc] #apply similarity check for matching lemma for question words and context words
    # term frequency in document
    counter_ = collections.Counter(context_tokens_lower) #number of times each word is repeating in ocntext
    total = len(context_tokens_lower) #Total number of words in context
    context_tf = [counter_[w] / total for w in context_tokens_lower] #Each word in context frequency probability 
    context_features = list(zip(match_origin, match_lower, match_lemma, context_tf)) #Zip all the features match_origin, match_lower, match_lemma, probability
    if not wv_cased:
        context_tokens = context_tokens_lower
        question_tokens = question_tokens_lower
    return (id_, context_tokens, context_features, context_tags, context_ents,
            question_tokens, context, context_token_span) + row[3:]  #Adding all these alogn with answer,answer_start,answer_end

Building annotations to training and dev data.

In [0]:
with Pool(args.threads, initializer=init) as p:
        annotate_ = partial(annotate, wv_cased=args.wv_cased)
        train = list(tqdm(p.imap(annotate_, train, chunksize=args.batch_size), total=len(train), desc='train'))
        dev = list(tqdm(p.imap(annotate_, dev, chunksize=args.batch_size), total=len(dev), desc='dev  '))

dev  : 100%|██████████| 10570/10570 [05:02<00:00, 34.98it/s]


In [0]:
def index_answer(row):
    token_span = row[-4] #Context tokens span
    starts, ends = zip(*token_span) #starting index of token is stored in starts and ending index of span is stored in ends
    answer_start = row[-2]
    answer_end = row[-1]
    try:
        return row[:-3] + (starts.index(answer_start), ends.index(answer_end)) #train till context token spans is added and index of answer start in starts and index of anser end in ends
    except ValueError:
        return row[:-3] + (None, None)

In [0]:
train = list(map(index_answer, train))
initial_len = len(train)
train = list(filter(lambda x: x[-1] is not None, train)) #Removing any empty features like answer is not present in starts and ends
log.info('drop {} inconsistent samples.'.format(initial_len - len(train)))
log.info('tokens generated')

04/18/2020 04:24:27 drop 999 inconsistent samples.
04/18/2020 04:24:27 tokens generated


In [0]:
 # load vocabulary from word vector files
wv_vocab = set()
with open(args.wv_file) as f:
        for line in f:
            token = normalize_text(line.rstrip().split(' ')[0])
            wv_vocab.add(token)
log.info('glove vocab loaded.')

04/18/2020 05:02:52 glove vocab loaded.


In [0]:
def build_vocab(questions, contexts, wv_vocab, sort_all=False):
    """
    Build vocabulary sorted by global word frequency, or consider frequencies in questions first,
    which is controlled by `args.sort_all`.
    """
    if sort_all: 
        counter = collections.Counter(w for doc in questions + contexts for w in doc) #number of times words in question appeared and words in context appeared and their together count.
        vocab = sorted([t for t in counter if t in wv_vocab], key=counter.get, reverse=True) #Those question words present in glove vocab are stored as vocab
    else:
        counter_q = collections.Counter(w for doc in questions for w in doc)
        counter_c = collections.Counter(w for doc in contexts for w in doc)
        counter = counter_c + counter_q
        vocab = sorted([t for t in counter_q if t in wv_vocab], key=counter_q.get, reverse=True)
        vocab += sorted([t for t in counter_c.keys() - counter_q.keys() if t in wv_vocab],
                        key=counter.get, reverse=True)
    vocab.insert(0, "<PAD>")
    vocab.insert(1, "<UNK>")
    return vocab, counter

In [0]:
 # build vocabulary
    full = train + dev
    #row[5] means question_tokens and row[1] means context_tokens
    vocab, counter = build_vocab([row[5] for row in full], [row[1] for row in full], wv_vocab, args.sort_all)
    total = sum(counter.values()) #Total occurences of question words
    matched = sum(counter[t] for t in vocab) #Total occurences of words in vocab
    log.info('vocab coverage {1}/{0} | OOV occurrence {2}/{3} ({4:.4f}%)'.format(
        len(counter), len(vocab), (total - matched), total, (total - matched) / total * 100))
    #Here row[3] means context POS tags
    counter_tag = collections.Counter(w for row in full for w in row[3]) #how many times each pos tag occured along with their frequencies
    vocab_tag = sorted(counter_tag, key=counter_tag.get, reverse=True) #Sorting of pos tags based on their frequency
    #Here row[4] means context NER tags
    counter_ent = collections.Counter(w for row in full for w in row[4]) #how many times each NER type occured along with their frequencies
    vocab_ent = sorted(counter_ent, key=counter_ent.get, reverse=True) #sorting of NER tags based on their frequencies
    #enumerate all words in vocab (<PAD>,<UNK>), vocab_tags and vocab_ent 
    w2id = {w: i for i, w in enumerate(vocab)}
    tag2id = {w: i for i, w in enumerate(vocab_tag)}
    ent2id = {w: i for i, w in enumerate(vocab_ent)}
    log.info('Vocabulary size: {}'.format(len(vocab)))
    log.info('Found {} POS tags.'.format(len(vocab_tag)))
    log.info('Found {} entity tags: {}'.format(len(vocab_ent), vocab_ent))

04/18/2020 05:05:32 vocab coverage 91590/111017 | OOV occurrence 110079/14779333 (0.7448%)
04/18/2020 05:05:36 Vocabulary size: 91590
04/18/2020 05:05:36 Found 50 POS tags.
04/18/2020 05:05:36 Found 19 entity tags: ['', 'ORG', 'DATE', 'PERSON', 'GPE', 'NORP', 'CARDINAL', 'LOC', 'FAC', 'WORK_OF_ART', 'EVENT', 'PERCENT', 'QUANTITY', 'ORDINAL', 'MONEY', 'LAW', 'PRODUCT', 'TIME', 'LANGUAGE']


row
0:id_,1 :context_tokens, 2:context_features, 3:context_tags, 4:context_ents, 5:question_tokens,6: context,7: context_token_span,8: starts.index(answer_start.index),9:ends.index(sanswers_end.index)

In [0]:
def to_id(row, w2id, tag2id, ent2id, unk_id=1):
    context_tokens = row[1]
    context_features = row[2]
    context_tags = row[3]
    context_ents = row[4]
    question_tokens = row[5]
    question_ids = [w2id[w] if w in w2id else unk_id for w in question_tokens]
    context_ids = [w2id[w] if w in w2id else unk_id for w in context_tokens]
    tag_ids = [tag2id[w] for w in context_tags]
    ent_ids = [ent2id[w] for w in context_ents]
    return (row[0], context_ids, context_features, tag_ids, ent_ids, question_ids) + row[6:]



row[0] --> id_, context_ids, context_features, tag_ids, ent_ids, question_ids, row[6:] --> context, context_token_span, starts.index(answer_start.index),9:ends.index(sanswers_end.index)



In [0]:
to_id_ = partial(to_id, w2id=w2id, tag2id=tag2id, ent2id=ent2id)
train = list(map(to_id_, train))
dev = list(map(to_id_, dev))
log.info('converted to ids.')

04/18/2020 05:07:07 converted to ids.


In [0]:
vocab_size = len(vocab) #178
embeddings = np.zeros((vocab_size, args.wv_dim)) #(178,300)
embed_counts = np.zeros(vocab_size) #(178,)
embed_counts[:2] = 1  # PADDING & UNK #(178,)

In [0]:
with open(args.wv_file) as f:
        for line in f:
            elems = line.rstrip().split(' ')
            token = normalize_text(elems[0])
            if token in w2id:
                word_id = w2id[token]
                embed_counts[word_id] += 1
                embeddings[word_id] += [float(v) for v in elems[1:]]
embeddings /= embed_counts.reshape((-1, 1))
log.info('got embedding matrix.')

04/18/2020 05:09:34 got embedding matrix.


In [0]:
meta = {
        'vocab': vocab, #All Vocabulary
        'vocab_tag': vocab_tag, #All vocab POS tags
        'vocab_ent': vocab_ent, #All vocab NER tags
        'embedding': embeddings.tolist(), #thier respective embeddings
        'wv_cased': args.wv_cased,
    }

In [0]:
# train: id, context_id, context_features, tag_id, ent_id,
#        question_id, context, context_token_span, answer_start, answer_end
# dev:   id, context_id, context_features, tag_id, ent_id,
#        question_id, context, context_token_span, answer
with open('meta.msgpack', 'wb') as f:
        msgpack.dump(meta, f)
result = {
        'train': train,
        'dev': dev
        }

In [0]:
with open('data.msgpack', 'wb') as f:
        msgpack.dump(result, f)

Sample size is defined for any debuggin gprocess. Here, we are not assigning smaplesize.

In [0]:
if args.sample_size:
        sample = {
            'train': train[:args.sample_size],
            'dev': dev[:args.sample_size]
        }
        with open('SQuAD/sample.msgpack', 'wb') as f:
            msgpack.dump(sample, f)
log.info('saved to disk.')

04/18/2020 05:11:28 saved to disk.
